<center><h1 style="color:grey;"> Tokenizador de textos </h1></center>

In [1]:
from IPython.display import clear_output
import gc
import psutil
import pickle
from math import log
print(gc.collect())
print("Memoria:", psutil.virtual_memory())

7
Memoria: svmem(total=12649299968, available=7030190080, percent=44.4, used=5619109888, free=7030190080)


In [2]:
import importlib
import dfrtokenuniverse.splitter
import dfrtokenuniverse.word_inventory
import dfrtokenuniverse.constantes

# Recarga librerías propias

In [5]:
importlib.reload(dfrtokenuniverse.splitter)
importlib.reload(dfrtokenuniverse.word_inventory)
importlib.reload(dfrtokenuniverse.constantes)
from dfrtokenuniverse.splitter import TextSplitter
from dfrtokenuniverse.constantes import KDfrNlp
K = KDfrNlp()
splitter = TextSplitter()

# Diccionario de Tokens

In [7]:
tokens_path = r"D:\datos\tokens_ttkn_id_v20241208.pkl"
print(f"Leyendo {tokens_path}")
with open(tokens_path, "rb") as file:
    tokens_dict = pickle.load(file)
print("Nada más leer:")
for ttkn in tokens_dict:
    print(ttkn, K.TTKN_DESC[ttkn], len(tokens_dict[ttkn]), [x for x in tokens_dict[ttkn]][:10])
tokens_dict[K.TTKN_SIS] = {token:K.MAIN_TOKEN_DICT[token] for token in K.MAIN_TOKEN_DICT}
print("Arreglado:")
for ttkn in tokens_dict:
    print(ttkn, K.TTKN_DESC[ttkn], len(tokens_dict[ttkn]), [x for x in tokens_dict[ttkn]][:10])

Leyendo D:\datos\tokens_ttkn_id_v20241208.pkl
Nada más leer:
6 Del Sistema 9 ['<', '|', 'v', 'a', 'i', 'd', '2', '|', '>']
1 Letras 14137 ['bar', 'bom', 'bo', 'deo', 'eo', 'de', 'ser', 'ivk', 'el', 'rr']
2 Separadores 359 [' , ', ' ,', '  ', '. ', ', ', '   ', ' .', ' . ', ' "', '".']
0 Dígitos 920 ['22', '20', '01', '00', '21', '12', '16', '38', '020', '521']
3 Símbolos 227 ['}}', '((', '))', '])', '>>', '}{', '})', ')}', ']}', '<{']
5 Desconocidos 5744 ['ɾ', 'ˈ', 'ː', 'ž', 'č', '\u200b', '«', '»', '“', '”']
4 Especiales 8 ['\t', '\t\t', '\t\t\t', '\x08', '\x07', '\x0c', '\r', '\x0b']
Arreglado:
6 Del Sistema 10 ['<|PAD|>', '<|UPPER|>', '<|LOWER|>', '<|CAP|>', '<|START|>', '<|TBC|>', '<|EOF|>', '<|UNK|>', '<|void1|>', '<|vaid2|>']
1 Letras 14137 ['bar', 'bom', 'bo', 'deo', 'eo', 'de', 'ser', 'ivk', 'el', 'rr']
2 Separadores 359 [' , ', ' ,', '  ', '. ', ', ', '   ', ' .', ' . ', ' "', '".']
0 Dígitos 920 ['22', '20', '01', '00', '21', '12', '16', '38', '020', '521']
3 Símbolos 227 ['}

In [8]:
# Recuperamos la informacion de tipo de token (ttkn) junto al token
ids_token = {}
MAX_TOKEN_ID = 0
for ttkn in tokens_dict:
    print(f"Vamos por ttkn={ttkn} ({K.TTKN_DESC[ttkn]})")
    for token in tokens_dict[ttkn]:
        token_id = tokens_dict[ttkn][token]
        ids_token[token_id] = {"token": token, "ttkn": ttkn}
        if token_id > MAX_TOKEN_ID:
            MAX_TOKEN_ID = tokens_dict[ttkn][token]
print(f"El máximo token_id es {MAX_TOKEN_ID} log: {round(log(MAX_TOKEN_ID, 10))} -> 10^x = {10**round(log(MAX_TOKEN_ID, 10))}")
TOKEN_DIVISOR = 10**round(log(MAX_TOKEN_ID, 10))
# Generamos los valores reales
token_id_real = {}
for token_id in ids_token:
    token_id_real[token_id] = round(token_id / TOKEN_DIVISOR, 7)
print(f"Generados {len(token_id_real)} tokens_id a reales")
def token_real_id(token_real):
    token_id = int(round(token_real, 7) * TOKEN_DIVISOR)
    if token_id in token_id_real:
        return token_id
    else:
        if token_id + 1 in token_id_real and token_id_real[token_id+1] == round(token_real,7):
            return token_id + 1
        else:
            return token_id - 1
#Samples
for token_id in [2, 51, 1230, 5132, 41235, 93256]:
    while token_id not in token_id_real:
        token_id += 1
        if token_id > MAX_TOKEN_ID:
            break
    treal = token_id_real[token_id]
    print(token_id, treal, token_real_id(treal), ids_token[token_id])

Vamos por ttkn=6 (Del Sistema)
Vamos por ttkn=1 (Letras)
Vamos por ttkn=2 (Separadores)
Vamos por ttkn=0 (Dígitos)
Vamos por ttkn=3 (Símbolos)
Vamos por ttkn=5 (Desconocidos)
Vamos por ttkn=4 (Especiales)
El máximo token_id es 95743 log: 5 -> 10^x = 100000
Generados 21405 tokens_id a reales
2 2e-05 2 {'token': '<|LOWER|>', 'ttkn': 6}
51 0.00051 51 {'token': '\t\t', 'ttkn': 4}
1230 0.0123 1230 {'token': '","', 'ttkn': 2}
5132 0.05132 5132 {'token': '$)', 'ttkn': 3}
41235 0.41235 41235 {'token': 'ner', 'ttkn': 1}
93256 0.93256 93256 {'token': 'ṯ', 'ttkn': 5}


In [97]:
def tokeniza(n_grama, ttkn):
    global tokens_dict
    tokenizado = []
    for token in tokens_dict[ttkn]:
        if token in n_grama:
            for subn in n_grama.split(token):
                if subn and subn > "":
                    tokenizado.extend(tokeniza(subn, ttkn))
                tokenizado.append(tokens_dict[ttkn][token])
            tokenizado.pop(-1)
            break
    return tokenizado
# Test
print(tokeniza("hola", 1))
print([ids_token[token_id]["token"] for token_id in tokeniza("hola", 1)])

def detokeniza(tokenizado):
    global ids_token
    detokenizado = ""
    token_sis = ""
    for token_ir in tokenizado:
        
        token = ""
        try:
            if token_ir < 1.0:
                token_id = token_real_id(token_ir)
            else:
                token_id = token_ir
            if ids_token[token_id]["ttkn"] == K.TTKN_SIS:
                token_sis = ids_token[token_id]["token"]
            else:
                token = ids_token[token_id]["token"]
                if token_sis == K.UPPER_TOKEN:
                    token = "" + token.upper()
                elif token_sis == K.CAP_TOKEN:
                    token = token[0].upper() + token[1:]
                token_sis = ""
        except Exception as e:
            token = "!"
        detokenizado += token
    return detokenizado
# Test
print(detokeniza([3, 40999, 40014]))
print(detokeniza([1, 40999, 1, 40014]))


[40999, 40014]
['ho', 'la']
Hola
HOLA


# Ejemplo splitter

In [10]:
tokenizado = []
texto = "Esto es una, hay otra ¿no?\nPues esta es una más ¡SEGURO! 23587,33"
for ttkn, n_grama in splitter.split_by_type(texto, 7):
    valores = splitter.normalize_n_gram(n_grama)
    if len(valores) > 1:
        tokenizado.append(K.MAIN_TOKEN_DICT[valores[0]])
    tokenizado.extend(tokeniza(valores[-1], ttkn))
print(tokenizado)
print([token_id_real[x] for x in tokenizado])
print([ids_token[token_id]["token"] for token_id in tokenizado])
print(detokeniza(tokenizado))
print(gc.collect())
print("Memoria:", psutil.virtual_memory())

[3, 40061, 54104, 1329, 40109, 1329, 40018, 54103, 1004, 40152, 54111, 1329, 54104, 40142, 1329, 5078, 40180, 5079, 1338, 3, 54119, 40070, 54112, 1329, 40061, 54103, 1329, 40109, 1329, 40018, 54103, 1329, 40036, 1329, 5083, 1, 5076, 1329, 20059, 20187, 20916, 1335, 20072]
[3e-05, 0.40061, 0.54104, 0.01329, 0.40109, 0.01329, 0.40018, 0.54103, 0.01004, 0.40152, 0.54111, 0.01329, 0.54104, 0.40142, 0.01329, 0.05078, 0.4018, 0.05079, 0.01338, 3e-05, 0.54119, 0.4007, 0.54112, 0.01329, 0.40061, 0.54103, 0.01329, 0.40109, 0.01329, 0.40018, 0.54103, 0.01329, 0.40036, 0.01329, 0.05083, 1e-05, 0.05076, 0.01329, 0.20059, 0.20187, 0.20916, 0.01335, 0.20072]
['<|CAP|>', 'est', 'o', ' ', 'es', ' ', 'un', 'a', ', ', 'ha', 'y', ' ', 'o', 'tra', ' ', '¿', 'no', '?', '\n', '<|CAP|>', 'p', 'ue', 's', ' ', 'est', 'a', ' ', 'es', ' ', 'un', 'a', ' ', 'más', ' ', '¡', '<|UPPER|>', '!', ' ', '23', '58', '7', ',', '33']
Esto es una, hay otra ¿no?
Pues esta es una más ¡! 23587,33
448
Memoria: svmem(total=126492

# Recuperamos texto

In [8]:
import pickle
wiki_path = r"D:\datos\wiki_1909627_entradas.pkl"
print(f"Leyendo desde: {wiki_path}")
with open(wiki_path, "rb") as wiki_file:
    wiki_data = pickle.load(wiki_file)
print(gc.collect())
print("Memoria:", psutil.virtual_memory())

Leyendo desde: D:\datos\wiki_1909627_entradas.pkl
0
Memoria: svmem(total=12649299968, available=1524199424, percent=88.0, used=11125100544, free=1524199424)


In [9]:
print(f"Documentos en wiki: {len(wiki_data)}")

Documentos en wiki: 1909627


<center> <h2>Tokenizando Texto</h2> </center>

In [14]:
texto_tokenized = {}

In [15]:
saltar  = 0
recoge = 10000 # Entradas de la wikipedia con sus textos
limite = saltar + recoge

<h3>Realizamos varias pasadas para ir acumulando n-gramas</h3>
<hr/>

In [16]:
print(f"Recogemos {recoge} entradas a partir de {saltar}")
cont_entradas = 0
cont_tokens = 0
max_len_allowed=7
for entrada in wiki_data:
    if cont_entradas > saltar:
        texto_tokenized[cont_entradas] = []
        tokenizado = []
        for ttkn, n_grama in splitter.split_by_type(entrada, max_len_allowed):
            valores = splitter.normalize_n_gram(n_grama)
            if len(valores) > 1:
                tokenizado.append(K.MAIN_TOKEN_DICT[valores[0]])
            tokenizado.extend(tokeniza(valores[-1], ttkn))
        texto_tokenized[cont_entradas].append([token_id_real[x] for x in tokenizado])
        cont_tokens += len(tokenizado)
        for linea in wiki_data[entrada]:
            tokenizado = []
            for ttkn, n_grama in splitter.split_by_type(linea.decode("utf-8"), max_len_allowed):
                valores = splitter.normalize_n_gram(n_grama)
                if len(valores) > 1:
                    tokenizado.append(K.MAIN_TOKEN_DICT[valores[0]])
                tokenizado.extend(tokeniza(valores[-1], ttkn))
            texto_tokenized[cont_entradas].append([token_id_real[x] for x in tokenizado])
            cont_tokens += len(tokenizado)
        if cont_entradas > limite:
            break
        if cont_entradas % 1000 == 0:
            clear_output(True)
            print(f"Llevamos {cont_entradas} entradas de {limite} ({round(100*cont_entradas/limite, 2)}%)")
            print(f"Con {cont_tokens} tokens generados")
    cont_entradas += 1
salta = limite
limite += recoge

Llevamos 10000 entradas de 10000 (100.0%)
Con 16383634 tokens generados


<hr style="border: 1px dashed #aaa;"/>
<center><h2>Liberamos memoria</h2></center>

In [17]:
print("Memoria:", psutil.virtual_memory())
del wiki_data
print(gc.collect())
print("Memoria:", psutil.virtual_memory())

Memoria: svmem(total=12649299968, available=3535872000, percent=72.0, used=9113427968, free=3535872000)
187
Memoria: svmem(total=12649299968, available=3725950976, percent=70.5, used=8923348992, free=3725950976)


In [19]:
print(gc.collect())
print("Memoria:", psutil.virtual_memory())

0
Memoria: svmem(total=12649299968, available=7463751680, percent=41.0, used=5185548288, free=7463751680)


In [18]:
tokens_path = r"D:\datos\texto_tokenizado_v20241208.pkl"
print(f"Guardando {tokens_path}")
with open(tokens_path, "wb") as file:
    pickle.dump(texto_tokenized, file)

Guardando D:\datos\texto_tokenizado_v20241208.pkl


In [17]:
tokens_path = r"D:\datos\texto_tokenizado_v20241208.pkl"
print(f"Leyendo {tokens_path}")
with open(tokens_path, "rb") as file:
    texto_tokenized = pickle.load(file)

Leyendo D:\datos\texto_tokenizado_v20241208.pkl


<hr style="color: red; height: 2; border: 3px dotted;"/>
<center><h1 style="color: yellow;">Comienza la fiesta</h1></center>
<hr style="color: red; width: 50%;"/>

In [11]:

FIRST_DIG_TOKEN_R = 0
for token in tokens_dict[K.TTKN_DIG]:
    FIRST_DIG_TOKEN_R = token_id_real[tokens_dict[K.TTKN_DIG][token]]
    break
print(f"El primer caracter DIGITO está en el token-r = {FIRST_DIG_TOKEN_R}")
FIRST_LET_TOKEN_R = 0
for token in tokens_dict[K.TTKN_LET]:
    FIRST_LET_TOKEN_R = token_id_real[tokens_dict[K.TTKN_LET][token]]
    break
print(f"El primer caracter LETRA está en el token-r = {FIRST_LET_TOKEN_R}")
FIRST_UNK_TOKEN_R = 0
for token in tokens_dict[K.TTKN_UNK]:
    FIRST_UNK_TOKEN_R = token_id_real[tokens_dict[K.TTKN_UNK][token]]
    break
print(f"El primer caracter DESCONOCIDO está en el token-r = {FIRST_UNK_TOKEN_R}")


El primer caracter DIGITO está en el token-r = 0.2
El primer caracter LETRA está en el token-r = 0.4
El primer caracter DESCONOCIDO está en el token-r = 0.9


# Preparación de texto para entrenamiento predictivo

In [66]:
class GetNextToken():
    """Clase para manejar la generacion de datos de entrenamiento
    Controla una lista de listas de vectores organizada de la siquiente manera:
    - nivel 0: articulo
    - nivel 1: linea 0 = titulo
    - nivel 1: linea 1+ = contenido
    - nivel 2: token_r
    """
    def __init__(self):
        self.art_id = 1
        self.i_l = 0
        self.i_c = 0
        self.log = []
    def reset(self):
        self.art_id = 1
        self.i_l = 0
        self.i_c = 0
        self.log = []
    def take_next(self, tokenizado):
        global token_id_real, K
        if self.art_id == 1 or self.i_l == 0 or self.i_c == 0:
            if len(tokenizado[self.art_id][self.i_l]) > 1:
                next_token = [
                    token_id_real[K.MAIN_TOKEN_DICT[K.START_TOKEN]],
                    tokenizado[self.art_id][self.i_l][self.i_c]
                ]
            else:
                next_token = []
        else:
            self.log.append(f"(A) art_id={self.art_id}, i_l={self.i_l}, i_c={self.i_c}")
            if len(tokenizado[self.art_id][self.i_l]) > 1:
                next_token = [tokenizado[self.art_id][self.i_l][self.i_c]]
            else:
                next_token = []
        self.i_c += 1
        if self.i_c >= len(tokenizado[self.art_id][self.i_l]):
            self.i_c = 0
            self.i_l += 1
            if self.i_l >= len(tokenizado[self.art_id]):
                self.log = [f"(B) art_id={self.art_id}, i_l={self.i_l}, i_c={self.i_c}"]
                self.i_l = 0
                self.art_id += 1
                if self.art_id >= len(tokenizado):
                    self.log = []
                    self.art_id = 1
                    next_token.append(token_id_real[K.MAIN_TOKEN_DICT[K.EOF_TOKEN]])
                else:
                    self.log.append(f"(C) art_id={self.art_id}, i_l={self.i_l}, i_c={self.i_c}")
                    next_token.append(token_id_real[K.MAIN_TOKEN_DICT[K.CONTINUE_TOKEN]])
            else:
                self.log.append(f"(D) art_id={self.art_id}, i_l={self.i_l}, i_c={self.i_c}")
                next_token.append(token_id_real[K.MAIN_TOKEN_DICT[K.CONTINUE_TOKEN]])
        return next_token
# TEst
gnt = GetNextToken()
for i in range(5):
    print(gnt.take_next(texto_tokenized), end=", ")
gnt.reset()
print(gnt.take_next(texto_tokenized))
gnt.reset()



[4e-05, 3e-05], [4e-05, 0.40001], [4e-05, 0.4], [4e-05, 0.54101], [4e-05, 0.40004], [4e-05, 3e-05]


## Generación de datos
El objetivo es aprender a predecir el siguiente token dados unos de partida.
- Features: vector de tokens
- Targets: vector de tokens corrido a la izquierda más un token nuevo

La característica principal de este set de datos es que rellena con ceros por la izquierda para:
- Entrenar mejor las neuronas más cercanas al nuevo token
- Favorecer que las neuronas menos usadas queden lejos del rango de cálculo

In [126]:
NUM_CONTEXT_TOKENS = 50
data_limit = 10000
min_info = 5 # numero de tokens minimo para entrenar
min_ceros = data_limit - min_info
X = []
Y = []
pad_token_r = token_id_real[K.MAIN_TOKEN_DICT[K.PAD_TOKEN]]
eof_token_r = token_id_real[K.MAIN_TOKEN_DICT[K.EOF_TOKEN]]
cont_token_r = token_id_real[K.MAIN_TOKEN_DICT[K.CONTINUE_TOKEN]]
print(f"El token <|PAD|> es : {pad_token_r}")
gnt.reset()
x_vector = [pad_token_r]*NUM_CONTEXT_TOKENS
y_vector = [pad_token_r]*NUM_CONTEXT_TOKENS
token_actu = gnt.take_next(texto_tokenized)
for token_r in token_actu:
    x_vector = y_vector.copy()
    y_vector = y_vector[1:] + [token_r]
cont_data = 1
# El proceso termina con fin de datos o con fin de seccion y limite de datos
while token_actu[-1] != eof_token_r and not (cont_data >= data_limit and token_actu[-1] == cont_token_r):
    # Si el token reciente no es fin de fichero, corremos a la izquierda posicion e incluimos el nuevo
    token_actu = gnt.take_next(texto_tokenized)
    for token_r in token_actu:
        x_vector = y_vector.copy()
        y_vector = y_vector[1:] + [token_r]
        # Solo generamos cuando un minimo de informacion
        if x_vector.count(0.0) < min_ceros:
            X.append(x_vector)
            Y.append(y_vector)
    # Si el token reciente es fin de algo, vaciamos la respuesta ordenadamente
    if token_actu[-1] in [cont_token_r, eof_token_r]:
        for token_r in token_actu:
            x_vector = y_vector.copy()
            y_vector = y_vector[1:] + [token_r]
            # Solo generamos cuando un minimo de informacion
            if x_vector.count(0.0) < min_ceros:
                X.append(x_vector)
                Y.append(y_vector)
        # Y empezamos de nuevo
        x_vector = [pad_token_r]*NUM_CONTEXT_TOKENS
        y_vector = [pad_token_r]*NUM_CONTEXT_TOKENS
    cont_data += 1
print("Finalizado! con límite de {data_limit} para vectores de {NUM_CONTEXT_TOKENS}")
print(f"Tenemos {len(X)} features y {len(Y)} targets")

El token <|PAD|> es : 0.0
Finalizado! con límite de {data_limit} para vectores de {NUM_CONTEXT_TOKENS}
Tenemos 12330 features y 12330 targets


### Sample training data

In [127]:
for i in range(8):
    print("X:", detokeniza(X[i]), X[i])
    print("Y:", detokeniza(Y[i]), Y[i])
    print()

X:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4e-05, 3e-05]
Y:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4e-05, 3e-05, 4e-05]

X:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4e-05, 3e-05, 4e-05]
Y: bom [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

# Red a entrenar

In [79]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

### Una red autoencoder
Porque, además de entrenamiento predictivo, queremos obtener un resultado semántico intermedio como embeddings.

In [128]:
# Arquitectura de la red
class Autoencoder(nn.Module):
    def __init__(self, input_size=100, encoder_size=1500, encode_layers=[64, 128], generative_layers=[128, 64]):
        super(Autoencoder, self).__init__()
        # Encoder
        encoder_layers = []
        current_size = input_size
        for hidden_size in encode_layers:
            encoder_layers.append(nn.Linear(current_size, hidden_size))
            encoder_layers.append(nn.ReLU())
            current_size = hidden_size
        encoder_layers.append(nn.Linear(current_size, encoder_size))  # Encoder layer
        encoder_layers.append(nn.Sigmoid())
        self.encoder = nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        current_size = encoder_size
        for hidden_size in reversed(generative_layers):
            decoder_layers.append(nn.Linear(current_size, hidden_size))
            decoder_layers.append(nn.ReLU())
            current_size = hidden_size
        decoder_layers.append(nn.Linear(current_size, input_size))  # Output layer
        decoder_layers.append(nn.Sigmoid())
        self.decoder = nn.Sequential(*decoder_layers)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Entrenamiento
def train_autoencoder(model, dataloader, epochs=50, lr=0.001):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in dataloader:
            inputs, targets = batch
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader):.6f}")

# Dividir la red en encoder y decoder
def split_model(model):
    encoder = model.encoder # Este modulo genera embeddings
    decoder = model.decoder
    return encoder, decoder

## Instanciado del modelo

In [129]:
# Datos de entrenamiento
batch_size = 16
dataset = TensorDataset(
    torch.tensor(X, dtype=torch.float32),
    torch.tensor(Y, dtype=torch.float32)
)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Arquitectura del modelo
model = Autoencoder(
    input_size=len(X[0]),
    encoder_size=1500,  # embeddings
    encode_layers=[len(X[0]) // 3, len(X[0]) // 2],
    generative_layers=[len(X[0]) * 3 , len(X[0]) * 7, len(X[0]) * 5]
)

total_params = sum(p.numel() for p in model.parameters())
print(f"Número total de parámetros: {total_params}")

Número total de parámetros: 563541


## Entrenamiento

In [133]:
train_autoencoder(model, dataloader, epochs=20, lr=0.01)

Epoch 1, Loss: 0.031936
Epoch 2, Loss: 0.029767
Epoch 3, Loss: 0.029075
Epoch 4, Loss: 0.028683


KeyboardInterrupt: 

## Prueba

In [134]:
# Separar encoder y decoder
encoder, decoder = split_model(model)
#
# Datos de prueba
#
sample_text = "¿De qué manera puedo probar esto?"
tokenizado = []
for ttkn, n_grama in splitter.split_by_type(sample_text, 7):
    valores = splitter.normalize_n_gram(n_grama)
    if len(valores) > 1:
        tokenizado.append(K.MAIN_TOKEN_DICT[valores[0]])
    tokenizado.extend(tokeniza(valores[-1], ttkn))
input_raw = [token_id_real[x] for x in tokenizado]
X_test = [[0.0]*(NUM_CONTEXT_TOKENS - len(input_raw)) + input_raw]
print(len(X_test[0]), detokeniza(X_test[0]))
sample_input = torch.tensor(X_test, dtype=torch.float32)
#
# Inferencia
#
encoded_features = encoder(sample_input)
decoded_output = decoder(encoded_features)
#
# Resultado
#
print("Input:", sample_input[0].tolist())
print("Encoded (embeddings):", encoded_features[0].tolist())
print("Decoded:", decoded_output[0].tolist())
print("Texto:", detokeniza([x for x in decoded_output[0].tolist() if x >= 0.0]))

50 ¿De qué manera ruedo rrobar esto?
Input: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05077999830245972, 2.9999999242136255e-05, 0.40005001425743103, 0.013290000148117542, 0.4118799865245819, 0.5412300229072571, 0.013290000148117542, 0.40217000246047974, 0.5411700010299683, 0.4029799997806549, 0.013290000148117542, 0.5411900281906128, 0.40070000290870667, 0.4009400010108948, 0.013290000148117542, 0.5411900281906128, 0.40042001008987427, 0.4000000059604645, 0.013290000148117542, 0.4006099998950958, 0.5410400032997131, 0.05079000070691109]
Encoded (embeddings): [0.003197272541001439, 0.005459378473460674, 0.0003162634966429323, 0.0007794348057359457, 6.60949299344793e-05, 0.005031181965023279, 0.00026092425105161965, 0.0018234200542792678, 0.0018189524998888373, 0.9152073860168457, 0.0024097347632050514, 0.0001841401244746521, 0.00019020085164811462, 0.0055401199497282505, 0.0009540896862